https://github.com/alaakh42/Sentiment_Model_Template/blob/master/Sentiment_using_pretrained_models.ipynb

In [1]:
#pip install lightgbm
# pip install BeautifulSoup

import numpy as np
import pandas as pd
import re
import nltk
import gensim
from tqdm import tqdm
#from BeautifulSoup import BeautifulSoup
from bs4 import BeautifulSoup
import gc
import scipy

# Models For Experimentation
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, LinearSVR
from sklearn.naive_bayes import MultinomialNB, GaussianNB
#import lightgbm as lgb 
#import xgboost as xgb

# For Model Assessment
from sklearn.metrics import classification_report, accuracy_score, auc

# Data Split
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

from sklearn.datasets import load_files
from pyvi import ViTokenizer

In [13]:
# Load training data file

training_data = load_files(r"./data2", encoding="utf-8")
X, y = training_data.data, training_data.target

In [14]:
# Load file chứa các từ stop_words

stop_ws = []
with open("vnstopword.txt", encoding="utf-8") as f :
    text = f.read()
    for word in text.split('\n') :  #Tách ra mỗi dòng là 1 từ stopword riêng lẻ
        stop_ws.append(word)
    f.close()

In [15]:
# Tiền xử lý cho tập dữ liệu training
documents = []
sum_word = 0
for sen in range(0, len(X)):
    
    # remove stop word
    document = []
    for word in str(X[sen]).split() :
        sum_word = sum_word + 1         # Đếm tổng số từ trong tập dữ liệu
        if (word not in stop_ws) :
            if ("_" in word) or (word.isalpha() == True):
                document.append(word)
    #print(document)
    document = ' '.join(document)
    
    replace_list = {
        'òa': 'oà', 'óa': 'oá', 'ỏa': 'oả', 'õa': 'oã', 'ọa': 'oạ', 'òe': 'oè', 'óe': 'oé','ỏe': 'oẻ',
        'õe': 'oẽ', 'ọe': 'oẹ', 'ùy': 'uỳ', 'úy': 'uý', 'ủy': 'uỷ', 'ũy': 'uỹ','ụy': 'uỵ', 'uả': 'ủa',
        'ả': 'ả', 'ố': 'ố', 'u´': 'ố','ỗ': 'ỗ', 'ồ': 'ồ', 'ổ': 'ổ', 'ấ': 'ấ', 'ẫ': 'ẫ', 'ẩ': 'ẩ',
        'ầ': 'ầ', 'ỏ': 'ỏ', 'ề': 'ề','ễ': 'ễ', 'ắ': 'ắ', 'ủ': 'ủ', 'ế': 'ế', 'ở': 'ở', 'ỉ': 'ỉ',
        'ẻ': 'ẻ', 'àk': u' à ','aˋ': 'à', 'iˋ': 'ì', 'ă´': 'ắ','ử': 'ử', 'e˜': 'ẽ', 'y˜': 'ỹ', 'a´': 'á',
        ' okey ': ' ok ', 'ôkê': ' ok ', 'oki': ' ok ', ' oke ':  ' ok ',' okay':' ok ','okê':' ok ',
        ' tks ': u' cám ơn ', 'thks': u' cám ơn ', 'thanks': u' cám ơn ', 'ths': u' cám ơn ', 'thank': u' cám ơn ',
        ' kg ': u' không ','not': u' không ', u' kg ': u' không ', ' k ': u' không ',' kh ':u' không ',' kô ':u' không ',' hok ':u' không ',' kp ': u' không phải ',u' kô ': u' không ', ' ko ': u' không ', u' ko ': u' không ', u' k ': u' không ', 'khong': u' không ', u' hok ': u' không ',' hong ':u' không',
        'vs': u' với ', 'wa': ' quá ', 'wá': u' quá', 'j': u' gì ', '“': ' ',' đx ': u' được ', 'dk': u' được ', 'dc': u' được ', 'đk': u' được ',
        'đc': u' được ',' thick ': u' thích ', 'store': u' cửa hàng ',
        'shop': u' cửa hàng ', 'sp': u' sản phẩm ', 'gud': u' tốt ','god': u' tốt ','wel done':' tốt ', 'good': u' tốt ', 'gút': u' tốt ',
        ' sấu ': u' xấu ','gut': u' tốt ', u' tot ': u' tốt ', u' nice ': u' tốt ', 'perfect': 'rất tốt', 'bt': u' bình thường ',
        'time': u' thời gian ', 'qá': u' quá ', u' ship ': u' giao hàng ', u' m ': u' mình ', u' mik ': u' mình ',
        'date': u' hạn sử dụng ', 'hsd': u' hạn sử dụng ','quickly': u' nhanh ', 'quick': u' nhanh ','fast': u' nhanh ','delivery': u' giao hàng ',u' síp ': u' giao hàng ',
        'beautiful': u' đẹp tuyệt vời ', u' tl ': u' trả lời ', u' r ': u' rồi ', u' shopE ': u' cửa hàng ',u' order ': u' đặt hàng ',
        'chất lg': u' chất lượng ',u' sd ': u' sử dụng ',u' dt ': u' điện thoại ',u' nt ': u' nhắn tin ',u' tl ': u' trả lời ',u' sài ': u' xài ',u'bjo':u' bao giờ ',
        'thik': u' thích ',u' sop ': u' cửa hàng ', ' fb ': ' facebook ', ' face ': ' facebook ', ' very ': u' rất ',u'quả ng ':u' quảng  ',
        ' dep ': u' đẹp ',u' xau ': u' xấu ','delicious': u' ngon ', u'hàg': u' hàng ', u'qủa': u' quả ','iu': u' yêu ','pv':u' phục vụ','nv':u' nhân viên','nc':u' nước',
        u' hó ':u' khó',u' hắc ':u' khắc ','_': ' ', u' ng ':u' người '}

    for k, v in replace_list.items():
        document = document.replace(k, v)

    # Loại bỏ Hashtag
    #document = re.sub(r'#([^\s]+)', '', str(X[sen]))
    document = re.sub(r'#([^\s]+)', '', document)
    # Loại bỏ các URL
    document = re.sub(r'http([^\s]+)', '', document)
    # Loại bỏ number có trong comment
    document = re.sub(r'\d+', '', document)
    # Loại bỏ các dấu câu, các ký tự đặc biệt (Emoji, icon cũng bị remove)
    document = re.sub(r'[^\w\s]','', document)
    # Chuyển chữ in hoa thành chữ thường
    document = document.lower()
    # Loại bỏ các ký tự dư thừa đơn lẻ (Nhiều ký tự đơn lẻ liền nhau kg xử lý được)
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    # Loại bỏ các Emoji, icon
    document = re.sub(r'[^\w\s,]', '', document)
    # Loại bỏ nhiều khoảng trắng dư thừa
     #document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Tokenization
    document = ViTokenizer.tokenize(document)
    
    document = ''.join(document)
    
    documents.append(document)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(documents, y, test_size=0.3, random_state=42)

In [17]:
import re, string
re_tok = re.compile(u'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')

def tokenize(s):
    return re_tok.sub(r' \1 ', s).split()


# Let's start by Glove Model

In [18]:
# load the GloVe vectors in a dictionary:
embeddings_index_glove = {}
f = open('glove.twitter.27B.200d.txt','r', encoding='utf-8')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index_glove[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index_glove))

1193514it [02:53, 6880.45it/s]

Found 1193514 word vectors.


In [19]:
# this function creates a normalized vector for the whole sentence
def sent2vec(s, embeddings_index):
    words = str(s).lower().encode().decode('utf-8')
    words = tokenize(words)
    words = [w for w in words if not w in stop_ws]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(200)
    return v / np.sqrt((v ** 2).sum())

In [20]:
#create sentence vectors using the above function for training and validation set

xtrain_glove = [sent2vec(x, embeddings_index_glove) for x in tqdm(X_train)]
xtest_glove = [sent2vec(x, embeddings_index_glove) for x in tqdm(X_test)]

xtrain_glove = np.array(xtrain_glove)
xtest_glove = np.array(xtest_glove)

100%|█████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:04<00:00, 614.51it/s]


After generating the features its time to build the model

In [21]:
# Xây dựng mô hình huấn luyện bằng NaiveBayes

import time
from sklearn.naive_bayes import BernoulliNB, GaussianNB

# Thêm biến này vào để kiểm tra thời gian huấn luyện
start_NB = time.time()


#classifier_NB = BernoulliNB(binarize=None)
classifier_NB = GaussianNB()
classifier_NB.fit(xtrain_glove, y_train)

end_NB = time.time()
time_NB = end_NB-start_NB

#Show ra thời gian huấn luyện của mô hình
#print("Thời gian huấn luyện: %s second" % str(end-start))
print("Thời gian huấn luyện: ", str(time_NB), " giây")

Thời gian huấn luyện:  0.028980731964111328  giây


In [95]:
# Xây dựng mô hình huấn luyện bằng Random Forest
import time
from sklearn.ensemble import RandomForestClassifier

# Thêm biến này vào để kiểm tra thời gian huấn luyện
start_RF = time.time()

classifier_RF = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier_RF.fit(xtrain_glove, y_train) 

end_RF = time.time()
time_RF = end_RF-start_RF

#Show ra thời gian huấn luyện của mô hình
#print("Thời gian huấn luyện: %s second" % str(end-start))
print("Thời gian huấn luyện: ", str(time_RF), " giây")

Thời gian huấn luyện:  425.9665598869324  giây


In [96]:
# Xây dựng mô hình huấn luyện bằng SVM
import time
from sklearn.svm import SVC

# Thêm biến này vào để kiểm tra thời gian huấn luyện
start_SVM = time.time()

classifier_SVM = SVC(kernel = 'linear', random_state = 0)
classifier_SVM.fit(xtrain_glove, y_train)


end_SVM = time.time()
time_SVM = end_SVM-start_SVM

#Show ra thời gian huấn luyện của mô hình
#print("Thời gian huấn luyện: %s second" % str(end-start))
print("Thời gian huấn luyện: ", str(time_SVM), " giây")

Thời gian huấn luyện:  166.216814994812  giây


In [97]:
# Xây dựng mô hình huấn luyện bằng Logistic Regression

from sklearn.linear_model import LogisticRegression
start = time.time()

classifier_LR = LogisticRegression(fit_intercept=True, n_jobs=1)
classifier_LR.fit(xtrain_glove, y_train)

end = time.time()

print("Thời gian huấn luyện Logistic Regression: ", str(end-start), " giây")

Thời gian huấn luyện Logistic Regression:  0.6775789260864258  giây


In [98]:
# Xây dựng mô hình huấn luyện bằng KNN

from sklearn.neighbors import KNeighborsClassifier

start = time.time()

classifier_KNN = KNeighborsClassifier(n_neighbors=5)
classifier_KNN.fit(xtrain_glove, y_train)

end = time.time()

print("Thời gian huấn luyện KNN: ", str(end-start), " giây")

Thời gian huấn luyện KNN:  2.043734073638916  giây


In [22]:
# Predicting Sentiment

y_pred_NB = classifier_NB.predict(xtest_glove)

#y_pred_RF = classifier_RF.predict(xtest_glove)

#y_pred_SVM = classifier_SVM.predict(xtest_glove)

#y_Pred_LR = classifier_LR.predict(xtest_glove)

#y_Pred_KNN = classifier_KNN.predict(xtest_glove)

In [23]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#print('Accuracy LogisticRegression: ', accuracy_score(y_test, y_Pred_LR))
print('Accuracy of Naive Bayes:',accuracy_score(y_test, y_pred_NB))
#print('Accuracy KNN: ', accuracy_score(y_test, y_Pred_KNN))
#print('Accuracy of Random Forest:',accuracy_score(y_test, y_pred_RF))
#print('Accuracy of SVM:',accuracy_score(y_test, y_pred_SVM))

Accuracy of Naive Bayes: 0.593
